# Decision Tree

This example shows how to use [SciKit-Learn](https://scikit-learn.org/stable/) to train a Decision Tree model on the Titanic dataset. Data is processed to increase the accuracy of the model. For a more detailed explanation of what is Decision Tree is, see [Decision Tree](../document/decision_tree.md).

## Imports

In [ ]:
import polars as pl
import seaborn as sns

from typing import Any

from matplotlib.axes import Axes
from matplotlib.text import Annotation
from numpy import ndarray
from polars import LazyFrame, DataFrame
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from seaborn import heatmap

## Process Data
Apply the same processing to the training and testing data.

Access the data lazily.

In [ ]:
train_data: LazyFrame = pl.scan_csv("data/train.csv", has_header=True)
test_data: LazyFrame = pl.scan_csv("data/test.csv", has_header=True)

Define a map of titles to integers. The title data was extracted from the _Name_ column in the training and testing data.

In [ ]:
title_map: dict[str, int] = {
    # Common Titles
    "Mr": 1,
    "Ms": 2,
    "Mrs": 2,
    "Mme": 2,
    "Master": 3,
    "Miss": 4,
    "Mlle": 4,
    # Distinguished Titles
    "Capt": 5,
    "Col": 5,
    "Dr": 5,
    "Major": 5,
    "Rev": 5,
    # Royal Titles
    "Countess": 6,
    "Don": 6,
    "Dona": 6,
    "Jonkheer": 6,
    "Lady": 6,
    "Sir": 6
}

### Features

Calculate the mean ages for records that do not have an age. Calculate the _mean_ age for "Master."s, "Miss."es, "Mr."s, and "Mrs."es. For other titles, use the mean age of the entire dataset. Titles are embedded in the _Name_ column.

> ![note]
> The following code is pseudo-code. It may not be executable.

```python

The ages was calculated roughly as follows:

```python
all_data = pl.concat(
    pl.read_csv("data/train.csv"),
    pl.read_csv("data/test.csv"),
)

mean_miss_age: float = all_data.filter(pl.col("Name").str.contains("Miss.")).get_column("Age").mean()
mean_master_age: float = all_data.filter(pl.col("Name").str.contains("Master.")).get_column("Age").mean()
mean_mrs_age: float = all_data.filter(pl.col("Name").str.contains("Mrs.")).get_column("Age").mean()
mean_mr_age: float = all_data.filter(pl.col("Name").str.contains("Mr.")).get_column("Age").mean()

mean_ages: DataFrame = all_data.with_columns(
    miss=pl.when(null_age_miss_filter).then(mean_miss_age).otherwise(pl.col("Age")),
    master=pl.when(null_age_master_filter).then(mean_master_age).otherwise(pl.col("Age")
    mrs=pl.when(null_age_mrs_filter).then(mean_mrs_age).otherwise(pl.col("Age")),
    mr=pl.when(null_age_mr_filter).then(mean_mr_age).otherwise(pl.col("Age")),
    mean=pl.col("Age").mean(),
)
```

In [ ]:
mean_miss_age = 21.5
mean_master_age = 4.5
mean_mrs_age = 35.5
mean_mr_age = 33.0

Prepare the features for training and inference:
+ Create a feature called `sku` from the 'Pclass' column
+ Create a feature called `n_family` by adding the 'SibSp' and 'Parch' columns
+ Create a feature called `title` by extracting the title from the 'Name' column and mapping it to an integer
+ Create a feature called `has_cabin` by checking if the 'Cabin' column is not null
+ Create a feature called `fare` by binning the 'Fare' column into 3 categories
+ Create a feature called `age` by using the calculated mean ages for the 'Age' column

In [ ]:
train_features: LazyFrame = train_data.select(
    sku=pl.col("Pclass").rank(method="dense"),
    n_family=pl.col("SibSp") + pl.col("Parch"),
    origin=pl.col("Embarked").fill_null(strategy="forward").rank(method="dense"),
    title=pl.col("Name")
        .str.extract(r",\s*(\w+)\.\s*")
        .replace_strict(title_map, default=max(title_map.values()) + 1, return_dtype=pl.UInt8),
    has_cabin=pl.col("Cabin").is_not_null().cast(pl.UInt8),
    fare=pl.when(pl.col("Fare").is_null(),)
        .then(pl.col("Fare").mean())
        .when(pl.col("Fare").le(7.91),).then(1)
        .when(pl.col("Fare").is_between(7.91, 14.454, closed='left'),).then(2)
        .when(pl.col("Fare").is_between(14.454, 31.0, closed='left'),).then(3)
        .otherwise(4).cast(pl.UInt8),
    age=pl.when((pl.col("Name").str.contains("Master")) & (pl.col("Age").is_null()),)
        .then(mean_master_age)
        .when((pl.col("Name").str.contains("Miss")) & (pl.col("Age").is_null()),)
        .then(mean_miss_age)
        .when((pl.col("Name").str.contains("Mrs")) & (pl.col("Age").is_null()),)
        .then(mean_mrs_age)
        .when((pl.col("Name").str.contains("Mr")) & (pl.col("Age").is_null()),)
        .then(mean_mr_age)
        .when(pl.col("Age").is_null(),)
        .then(pl.col("Age").mean())
        .otherwise(pl.col("Age")),
    # is father with family
    is_father=pl.when(pl.col("Name").str.contains("Mr") & (pl.col("Parch") + pl.col("SibSp") > 2),)
        .then(1)
        .otherwise(0).cast(pl.UInt8),
    # Child under 12 in family
    is_lt12fam=pl.when(pl.col("Age").lt(12) & pl.col("Parch") + pl.col("SibSp") > 1,)
        .then(1)
        .otherwise(0).cast(pl.UInt8),
)

test_features: LazyFrame = test_data.select(
    sku=pl.col("Pclass").rank(method="dense"),
    n_family=pl.col("SibSp") + pl.col("Parch"),
    origin=pl.col("Embarked").fill_null(strategy="forward").rank(method="dense"),
    title=pl.col("Name")
        .str.extract(r",\s*(\w+)\.\s*")
        .replace_strict(title_map, default=max(title_map.values()) + 1, return_dtype=pl.UInt8),
    has_cabin=pl.col("Cabin").is_not_null().cast(pl.UInt8),
    fare=pl.when(pl.col("Fare").is_null(),)
        .then(pl.col("Fare").mean())
        .when(pl.col("Fare").le(7.91),).then(1)
        .when(pl.col("Fare").is_between(7.91, 14.454, closed='left'),).then(2)
        .when(pl.col("Fare").is_between(14.454, 31.0, closed='left'),).then(3)
        .otherwise(4).cast(pl.UInt8),
    age=pl.when((pl.col("Name").str.contains("Master")) & (pl.col("Age").is_null()),)
        .then(mean_master_age)
        .when((pl.col("Name").str.contains("Miss")) & (pl.col("Age").is_null()),)
        .then(mean_miss_age)
        .when((pl.col("Name").str.contains("Mrs")) & (pl.col("Age").is_null()),)
        .then(mean_mrs_age)
        .when((pl.col("Name").str.contains("Mr")) & (pl.col("Age").is_null()),)
        .then(mean_mr_age)
        .when(pl.col("Age").is_null(),)
        .then(pl.col("Age").mean())
        .otherwise(pl.col("Age")),
    # is father with family
    is_father=pl.when(pl.col("Name").str.contains("Mr") & (pl.col("Parch") + pl.col("SibSp") > 2),)
        .then(1)
        .otherwise(0).cast(pl.UInt8),
    # Child under 12 in family
    is_lt12fam=pl.when(pl.col("Age").lt(12) & pl.col("Parch") + pl.col("SibSp") > 1,)
        .then(1)
        .otherwise(0).cast(pl.UInt8),
)

train_output: LazyFrame = train_data.select(y=pl.col("Survived"))

### View Some of the Processed Data

In [ ]:
sample_train_features: DataFrame = train_features.collect().head(15)
sample_train_features

In [ ]:
sample_test_features: DataFrame = test_features.collect().head(15)
sample_test_features

### Train the Decision Tree

In [ ]:
X: ndarray[Any, Any] = train_features.collect().to_numpy()
y: ndarray[Any, Any] = train_output.collect().to_numpy()

X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=73)

In [ ]:
# Train the decision tree classifier
clf = DecisionTreeClassifier(max_depth=3)
clf: DecisionTreeClassifier = clf.fit(X_train, y_train)

### Plot the Decision Tree

In [ ]:
sns.set_theme(font_scale=2, rc={"figure.figsize": (40, 20)})
tree_plot: list[Annotation] = tree.plot_tree(
    clf,
    feature_names=train_features.collect_schema().names(),
    class_names=True,
    filled=True,
    rounded=True,
    proportion=True
)

### Export the Decision Tree

In [ ]:
# Export the decision tree to a dot file
# dot_data = tree.export_graphviz(
#     clf,
#     out_file="resource/decision-tree.dot",
#     feature_names=train_features.columns,
#     class_names=["0", "1"],
#     filled=True,
#     rounded=True,
#     special_characters=True,
# )


### Evalue the Model

In [ ]:
# Evaluate the model
y_pred: ndarray = clf.predict(X_validate)
accuracy: float = accuracy_score(y_validate, y_pred)
precision: float = precision_score(y_validate, y_pred)
recall: float = recall_score(y_validate, y_pred)
f1: float = f1_score(y_validate, y_pred)

print(f"Accuracy: {100 * accuracy:.2f}%")
print(f"Precision: {100 * precision:.2f}%")
print(f"Recall: {100 * recall:.2f}%")
print(f"F1: {100 * f1:.2f}%")

### Plot the Confusion Matrix

In [ ]:
sns.axes_style(rc={"xtick.top": True, "axes.spines.top": True})

confusion: ndarray = confusion_matrix(y_validate, y_pred)

plot: Axes=heatmap(confusion, annot=True, fmt="d", xticklabels=["Foundered", "Survived"], yticklabels=["Foundered", "Survived"])

### Generate Prediction List

In [ ]:
predictions: ndarray = clf.predict(test_features.collect().to_numpy())

predition_list: DataFrame = test_data.select(
    PassengerId=pl.col("PassengerId"),
    Survived=pl.Series(predictions)
).collect()

predition_list.write_csv("data/decision_tree_predictions.csv")

### Compare the Predictions with the Ground Truth

In [ ]:
source = pl.read_csv('data/decision_tree_predictions.csv')
target = pl.read_csv('data/gender_submission.csv')

y_source = source['Survived']
y_target = target['Survived']

num_differences = (y_source != y_target).sum()
num_difference_percentage = (num_differences / len(y_source)) * 100
num_difference_percentage



----
Go back to [index](_index.ipynb).